In [3]:
from langchain import LLMChain
from langchain.agents import initialize_agent, Tool
from langchain.llms import Ollama

In [ ]:
llm = Ollama(model="llama3.2:1b")

In [51]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="llama3.2:1b", format="json")

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

model.invoke("what is the weather in Boston?")

AIMessage(content="It's currently sunny in Boston with a high of 22 degrees Celsius and a low of 10 degrees Celsius. ", additional_kwargs={}, response_metadata={}, id='run-31c874b7-89f2-4329-a664-f9a7fd2e573c-0')

In [48]:
from typing import TypedDict, List
from pydantic import BaseModel, Field
from langchain.agents import Tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions

def sumar_numeros(a: int, b: int) -> int:
    """Suma dos números y devuelve el resultado."""
    return a + b

class SumaInput(BaseModel):
    """Define los parámetros que recibirá la función sumar_numeros."""
    a: int = Field(description="El primer número.")
    b: int = Field(description="El segundo número.")

# Definir la función en el formato que espera OllamaFunctions
suma_function = {
    "name": "sumar_numeros",
    "description": "Suma dos números y devuelve el resultado.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "integer", "description": "El primer número"},
            "b": {"type": "integer", "description": "El segundo número"}
        },
        "required": ["a", "b"]
    }
}

# Crear el modelo
model = OllamaFunctions(
    model="llama3.2:1b",
    format="json"
)

# Vincular la función
model = model.bind_tools(
    tools=[suma_function],
    function_call={"name": "sumar_numeros"}
)

# Realizar la invocación
response = model.invoke("cuanto es 23444 mas 2828282?")
print(response)

content='{"result": 4569896}' additional_kwargs={} response_metadata={} id='run-471d9061-0c2b-42f5-a222-808516be8035-0'


In [ ]:

# Realizar la invocación
model.invoke("suma es 23444 mas 2828282?")

In [57]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [58]:
# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

In [61]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings
# Create embeddings for documents and store them in a vector store
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding = OllamaEmbeddings(
    model="llama3.2:1b",
),
)
retriever = vectorstore.as_retriever(k=4)

In [ ]:
from langchain_ollama import ChatOllama
# Initialize the LLM with Llama 3.1 model
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
)

In [ ]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

In [ ]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [ ]:
# Initialize the RAG application
rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "What is prompt engineering"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)